In [1]:
import os
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sps
import implicit
import pickle
from implicit.evaluation import train_test_split, precision_at_k, mean_average_precision_at_k



In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
    names=['user_id', 'movie_id', 'rating', 'timestamp'], 
    usecols=['user_id', 'movie_id', 'rating','timestamp'],
    engine='python')
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
# user_id_list = ratings['user_id'].values
# item_id_list = ratings['movie_id'].values
# rating_list = ratings['rating'].values
# timestamp_list = ratings['timestamp'].values

In [4]:
ratings['rating']=1
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,1,978300760
1,1,661,1,978302109
2,1,914,1,978301968
3,1,3408,1,978300275
4,1,2355,1,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,1,956704887
1000206,6040,562,1,956704746
1000207,6040,1096,1,956715648


In [5]:
df_user_movie_rating= ratings.pivot(
    index='user_id',
    columns='movie_id',
    values='rating'
).fillna(0)
df_user_movie_rating

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_user_movie_timestamp= ratings.pivot(
    index='user_id',
    columns='movie_id',
    values='timestamp'
).fillna(0)
df_user_movie_timestamp

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,978824268.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,978245916.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,956753920.0,0.0,956717530.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
import numpy as np

df_user_movie_rating_mtx = df_user_movie_rating.values
df_user_movie_timestamp_mtx = df_user_movie_timestamp.values

In [8]:
# df_user_movie_rating
df_user_movie_rating_mtx
print(df_user_movie_rating_mtx.shape[0])

6040


In [9]:
from IncrementalSparseMatrix import IncrementalSparseMatrix

In [10]:
import os
os.getcwd() 

'/Users/macpro/Recsys/Anaelle/Data_manager'

In [11]:
def split_data_on_timestamp(URM, URM_timestamp):

    URM_all=sps.csr_matrix(URM)
    URM_timestamp=sps.csr_matrix(URM_timestamp)

    n_rows, n_cols = URM_all.shape


    URM_train_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    URM_test_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    URM_validation_builder = IncrementalSparseMatrix(n_rows=n_rows, n_cols=n_cols)
    

    all_items = np.arange(0, n_cols, dtype=np.int)


    for user_index in range(URM_all.shape[0]):

        if user_index % 10000 == 0:
            print("split_data_on_sequence: user {} of {}".format(user_index, URM_all.shape[0]))

        start_pos = URM_all.indptr[user_index]
        end_pos = URM_all.indptr[user_index+1]

        user_profile = URM_all.indices[start_pos:end_pos]
        user_data = URM_all.data[start_pos:end_pos] 
#         implicit이므로 딱히 필요 없을지도..
        user_sequence = URM_timestamp.data[start_pos:end_pos]


        if len(user_profile) >= 3:

            # Test contain the first one, validation the second
            min_pos = np.argmax(user_sequence)

            venue_index = user_profile[min_pos]
            venue_data = user_data[min_pos] ##1

            URM_test_builder.add_data_lists([user_index], [venue_index], [venue_data])

            user_profile = np.delete(user_profile, min_pos)
            user_data = np.delete(user_data, min_pos)
            user_sequence = np.delete(user_sequence, min_pos)


            min_pos = np.argmax(user_sequence)

            venue_index = user_profile[min_pos]
            venue_data = user_data[min_pos]

            URM_validation_builder.add_data_lists([user_index], [venue_index], [venue_data])

            user_profile = np.delete(user_profile, min_pos)
            user_data = np.delete(user_data, min_pos)
            #user_sequence = np.delete(user_sequence, min_pos)


            URM_train_builder.add_data_lists([user_index]*len(user_profile), user_profile, user_data)


    URM_train = URM_train_builder.get_SparseMatrix()
    URM_validation = URM_validation_builder.get_SparseMatrix()
    URM_test = URM_test_builder.get_SparseMatrix()
#     URM_negative = URM_negative_builder.get_SparseMatrix()



    return URM_train, URM_validation, URM_test

In [12]:
URM_train, URM_validation, URM_test = split_data_on_timestamp(df_user_movie_rating_mtx,df_user_movie_timestamp_mtx)

split_data_on_sequence: user 0 of 6040


In [13]:
from Evaluator.Evaluator import EvaluatorHoldout
from TopPopular import TopPopular
from check_matrix import check_matrix

In [27]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[1, 5, 10, 20], exclude_seen=False)

In [28]:
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[1, 5, 10, 20], exclude_seen=False)

In [29]:
recommender = TopPopular(URM_train)
recommender.fit()
results_dict = evaluator_validation.evaluateRecommender(recommender)
print(results_dict)

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.69 sec. Users per second: 3581
CUTOFF: 1 - AUROC: 0.0034768, NDCG: 0.0034768, HR: 0.0034768, 
CUTOFF: 5 - AUROC: 0.0068709, NDCG: 0.0072885, HR: 0.0109272, 
CUTOFF: 10 - AUROC: 0.0114790, NDCG: 0.0107707, HR: 0.0218543, 
CUTOFF: 20 - AUROC: 0.0236929, NDCG: 0.0173972, HR: 0.0481788, 



--------------------------------------

In [19]:
from item_CF import *

In [47]:
recommender = item_CF(URM_train)
recommender.fit(feature_weighting = "TF-IDF", similarity='cosine', topK=30)

In [48]:
results_dict = evaluator_validation.evaluateRecommender(recommender)
results_dict

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.03 sec. Users per second: 2968


{1: {'AUROC': 0.006788079470198676,
  'NDCG': 0.006788079470198676,
  'HR': 0.006788079470198676},
 5: {'AUROC': 0.013451986754966887,
  'NDCG': 0.0145758263717424,
  'HR': 0.02218543046357616},
 10: {'AUROC': 0.023105224429727744,
  'NDCG': 0.020868322983482814,
  'HR': 0.041721854304635764},
 20: {'AUROC': 0.04134715928895085,
  'NDCG': 0.0299028220580311,
  'HR': 0.07798013245033113}}

In [22]:
recommender = item_CF(URM_train)
top = [10, 20, 30, 50, 100]

for i in top:
    recommender.fit(feature_weighting = "TF-IDF", similarity='cosine', topK=i)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)


EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.29 sec. Users per second: 2640
topk :  10
CUTOFF: 1 - AUROC: 0.0089404, NDCG: 0.0089404, HR: 0.0089404, 
CUTOFF: 5 - AUROC: 0.0149834, NDCG: 0.0160401, HR: 0.0230132, 
CUTOFF: 10 - AUROC: 0.0236571, NDCG: 0.0218270, HR: 0.0410596, 
CUTOFF: 20 - AUROC: 0.0421140, NDCG: 0.0309858, HR: 0.0774834, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.15 sec. Users per second: 2813
topk :  20
CUTOFF: 1 - AUROC: 0.0066225, NDCG: 0.0066225, HR: 0.0066225, 
CUTOFF: 5 - AUROC: 0.0141142, NDCG: 0.0151634, HR: 0.0233444, 
CUTOFF: 10 - AUROC: 0.0232524, NDCG: 0.0209681, HR: 0.0415563, 
CUTOFF: 20 - AUROC: 0.0421227, NDCG: 0.0302441, HR: 0.0784768, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.36 sec. Users per second: 2559
topk :  30
CUTOFF: 1 - AUROC: 0.0067881, NDCG: 0.0067881, HR: 0.0067881, 
CUTOFF: 5 - AUROC: 0.0134520, NDCG: 0.0145758, HR: 0.0221854, 
CUTOFF: 10 - AUROC: 0.0231052, NDCG: 0.0208683, HR: 0.0417219, 
CUTOFF: 20 - AUROC: 0.04

In [23]:
recommender = item_CF(URM_train)
top = [10, 20, 30, 50, 100]

for i in top:
    recommender.fit(feature_weighting = "BM25", similarity='cosine', topK=i)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)



EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.01 sec. Users per second: 2999
topk :  10
CUTOFF: 1 - AUROC: 0.0079470, NDCG: 0.0079470, HR: 0.0079470, 
CUTOFF: 5 - AUROC: 0.0154387, NDCG: 0.0173551, HR: 0.0269868, 
CUTOFF: 10 - AUROC: 0.0262693, NDCG: 0.0242638, HR: 0.0488411, 
CUTOFF: 20 - AUROC: 0.0468107, NDCG: 0.0334393, HR: 0.0855960, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.95 sec. Users per second: 3103
topk :  20
CUTOFF: 1 - AUROC: 0.0062914, NDCG: 0.0062914, HR: 0.0062914, 
CUTOFF: 5 - AUROC: 0.0138245, NDCG: 0.0160766, HR: 0.0263245, 
CUTOFF: 10 - AUROC: 0.0245769, NDCG: 0.0217506, HR: 0.0442053, 
CUTOFF: 20 - AUROC: 0.0447630, NDCG: 0.0322794, HR: 0.0864238, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.98 sec. Users per second: 3052
topk :  30
CUTOFF: 1 - AUROC: 0.0054636, NDCG: 0.0054636, HR: 0.0054636, 
CUTOFF: 5 - AUROC: 0.0118791, NDCG: 0.0133992, HR: 0.0213576, 
CUTOFF: 10 - AUROC: 0.0217807, NDCG: 0.0202975, HR: 0.0430464, 
CUTOFF: 20 - AUROC: 0.04

In [16]:
from P3alpha import *

In [41]:
recommender = P3alpha(URM_train)
recommender.fit(topK=50, alpha=0.5)

In [42]:
results_dict = evaluator_validation.evaluateRecommender(recommender)
results_dict

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.72 sec. Users per second: 3520


{1: {'AUROC': 0.004470198675496689,
  'NDCG': 0.004470198675496689,
  'HR': 0.004470198675496689},
 5: {'AUROC': 0.010306291390728476,
  'NDCG': 0.011777519661661805,
  'HR': 0.019370860927152317},
 10: {'AUROC': 0.019150110375275926,
  'NDCG': 0.017091262182652556,
  'HR': 0.035927152317880795},
 20: {'AUROC': 0.03583130010456606,
  'NDCG': 0.02546796931188233,
  'HR': 0.0695364238410596}}

In [38]:
# recommender = item_CF(URM_train+URM_validation)
# recommender.fit(feature_weighting = "TF_IDF")
recommender = P3alpha(URM_train+URM_validation)
recommender.fit(topK=50, alpha=0.5)

In [39]:
results_dict = evaluator_test.evaluateRecommender(recommender)
results_dict

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.83 sec. Users per second: 3295


{1: {'AUROC': 0.005298013245033113,
  'NDCG': 0.005298013245033113,
  'HR': 0.005298013245033113},
 5: {'AUROC': 0.011299668874172185,
  'NDCG': 0.012440455038815934,
  'HR': 0.01970198675496689},
 10: {'AUROC': 0.019922737306843265,
  'NDCG': 0.017711860173388033,
  'HR': 0.03609271523178808},
 20: {'AUROC': 0.035508888114325544,
  'NDCG': 0.025300063042352532,
  'HR': 0.06655629139072848}}

In [17]:
recommender = P3alpha(URM_train)

for i in range(10):
    recommender.fit(topK=50, alpha=i*0.1)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("alpha : ", i*0.1)
    print(results_dict)

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.81 sec. Users per second: 3333
alpha :  0.0
CUTOFF: 1 - AUROC: 0.0048013, NDCG: 0.0048013, HR: 0.0048013, 
CUTOFF: 5 - AUROC: 0.0114238, NDCG: 0.0128623, HR: 0.0210265, 
CUTOFF: 10 - AUROC: 0.0197940, NDCG: 0.0173393, HR: 0.0350993, 
CUTOFF: 20 - AUROC: 0.0342105, NDCG: 0.0242971, HR: 0.0630795, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.70 sec. Users per second: 3545
alpha :  0.1
CUTOFF: 1 - AUROC: 0.0046358, NDCG: 0.0046358, HR: 0.0046358, 
CUTOFF: 5 - AUROC: 0.0112169, NDCG: 0.0126275, HR: 0.0205298, 
CUTOFF: 10 - AUROC: 0.0192237, NDCG: 0.0171620, HR: 0.0349338, 
CUTOFF: 20 - AUROC: 0.0348466, NDCG: 0.0247058, HR: 0.0650662, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 2.16 sec. Users per second: 2791
alpha :  0.2
CUTOFF: 1 - AUROC: 0.0049669, NDCG: 0.0049669, HR: 0.0049669, 
CUTOFF: 5 - AUROC: 0.0106788, NDCG: 0.0119860, HR: 0.0192053, 
CUTOFF: 10 - AUROC: 0.0186718, NDCG: 0.0168267, HR: 0.0344371, 
CUTOFF: 20 - AUROC

- HR기준 최적 알파: 0.7

In [18]:
recommender = P3alpha(URM_train)
top = [20, 30, 50, 100]
for i in top:
    recommender.fit(topK=i, alpha=0.7)
    results_dict = evaluator_validation.evaluateRecommender(recommender)
    print("topk : ", i)
    print(results_dict)

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.71 sec. Users per second: 3542
topk :  20
CUTOFF: 1 - AUROC: 0.0051325, NDCG: 0.0051325, HR: 0.0051325, 
CUTOFF: 5 - AUROC: 0.0117550, NDCG: 0.0127760, HR: 0.0200331, 
CUTOFF: 10 - AUROC: 0.0202907, NDCG: 0.0179579, HR: 0.0360927, 
CUTOFF: 20 - AUROC: 0.0358749, NDCG: 0.0255262, HR: 0.0663907, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.72 sec. Users per second: 3521
topk :  30
CUTOFF: 1 - AUROC: 0.0046358, NDCG: 0.0046358, HR: 0.0046358, 
CUTOFF: 5 - AUROC: 0.0112583, NDCG: 0.0126576, HR: 0.0205298, 
CUTOFF: 10 - AUROC: 0.0198859, NDCG: 0.0173475, HR: 0.0350993, 
CUTOFF: 20 - AUROC: 0.0358400, NDCG: 0.0255528, HR: 0.0678808, 

EvaluatorHoldout: Processed 6040 ( 100.00% ) in 1.82 sec. Users per second: 3319
topk :  50
CUTOFF: 1 - AUROC: 0.0044702, NDCG: 0.0044702, HR: 0.0044702, 
CUTOFF: 5 - AUROC: 0.0109685, NDCG: 0.0121026, HR: 0.0195364, 
CUTOFF: 10 - AUROC: 0.0193893, NDCG: 0.0169993, HR: 0.0347682, 
CUTOFF: 20 - AUROC: 0.03